In [ ]:
import os
import pandas as pd
import numpy as np
from edgar import Company
import re
from matplotlib import pyplot as pp
import nltk as nltk

import pickle

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
from matplotlib import pyplot as plt

import os
os.chdir('/Users/brandon/Documents/UCL/COMP0087/CW2')

from time import time

# Full Edgar Access

## Load data and get unique ciks
These come from https://www.kaggle.com/finnhub/sec-filings?select=2021.QTR1.csv which has data back to '94!

In [ ]:
os.chdir('/Users/brandon/Documents/UCL/COMP0087/CW2/archive')

In [ ]:
read_files = [
    '2020.QTR1.csv',
    '2020.QTR2.csv',
    '2020.QTR3.csv',
    '2020.QTR4.csv',
]

In [ ]:
# pull data from finhub sec filings
ciks = []
for fn in read_files:
    frame = pd.read_csv('2020.QTR1.csv')
    ciks.append(frame[frame.form=='10-K'].loc[:, ['symbol', 'cik']])
ciks = pd.concat(ciks)

In [ ]:
# remove null data
ciks = ciks[~pd.isnull(ciks['symbol'])]
ciks = ciks[~pd.isnull(ciks['cik'])]

# multiple dupicate filings so drop these
ciks = ciks.drop_duplicates()

We now have ciks for all unique 10-K filings in 2020! There are about 4000

In [ ]:
ciks.sort_values('symbol')

In [ ]:
len(np.unique(ciks['cik'].values))

In [ ]:
# clearly some tickers have multiple ciks
len(np.unique(ciks['symbol'].values))

## Get business description data for ciks
We download data from EDGAR via package edgar using the ciks determined above. Then we look through the raw text to find the business description and pull it out.

In [ ]:
def find_sequence(seq, all_tokens):
    check_num = 0
    seq_loc = 0
    for ll, ss in enumerate(all_tokens):
        if ss.lower() in seq:
            check_num += 1
        else:
            check_num = 0
        # highly hacky as first location is in table of contents; this return second
        if check_num == len(seq):
            seq_loc = ll + 1
    return seq_loc

In [ ]:
i = 0
bds = {}
try_starts = [
    {'item', '1.', 'business'},
    {'item', '1.', 'our', 'business'},
    {'item', '1:', 'business'},
    {'item', '1.', 'business.'},
    {'item', '1', 'business'},
    {'item', '1.business'},
    
        ]
try_ends = [
    {'item', '1a.', 'risk', 'factors'},
    {'item', '1a:', 'risk', 'factors'},
    {'item', '1a.', 'risk', 'factors.'},
    {'item', '1a', 'risk', 'factors'},
        ]
max_tries = 5

In [ ]:
loop_over = [(x, y) for x, y in zip(ciks['symbol'].values, ciks['cik'].values)]
for ticker, cik in loop_over:
    print(f'Extracting {ticker}:{cik}')
    tt = 0
    while tt < max_tries:
        try:
            company = Company(ticker, str(cik))
            tree = company.get_all_filings(filing_type = "10-K")
            docs = Company.get_documents(tree, no_of_documents=5)
            break
        except:
            tt+=1
    if tt == max_tries:
        print(f'Edgar timeout for {ticker}')
        continue
    for doc in docs:
        fs_text = doc.text_content()
        # could tokenise better here
        # re.split(r'[ \t\n]+', fs_text) would split on any number of tabs, whitespaces, newlines
        fs_text = fs_text.replace('\t', ' ')
        fs_text = fs_text.replace('\n', ' ')
        tokenised = fs_text.split()
        
        for ts in try_starts:
            start_seq = find_sequence(ts, tokenised)
            if start_seq != 0:
                break
        for te in try_ends:
            end_seq = find_sequence(te, tokenised)
            if end_seq != 0:
                break
        checks = (start_seq != 0) & (end_seq != 0) & ((end_seq - start_seq) > 100)
        if checks:
            print(f'from {start_seq} to {end_seq} out of {len(tokenised)}')
            break
    if checks:
        bds[f'{ticker}:{cik}'] = tokenised[start_seq:end_seq]
    else:
        bds[f'{ticker}:{cik}'] = []


In [ ]:
pickle.dump(bds, open("bds_1.p", "wb"))

In [ ]:
os.chdir('/Users/brandon/Documents/UCL/COMP0087/CW2')
file1 = open("bds_1.txt","a")
for kk, vv in bds.items():
    file1.write(kk + '\n')
    file1.write(' '.join(vv)+ '\n')

file1.close()

In [ ]:
i = 0
for vv in bds.values():
    if len(vv) > 0:
        i += 1
i

In [ ]:
def extract_text(cik, doc_num):
    company = Company(str(cik), str(cik))
    tree = company.get_all_filings(filing_type = "10-K")
    docs = Company.get_documents(tree, no_of_documents=doc_num+1)
    fs_text = doc.text_content()
    fs_text = fs_text.replace('\t', ' ')
    fs_text = fs_text.replace('\n', ' ')
    tokenised = fs_text.split()
    print(' '.join(tokenised)[:100000])

In [ ]:
extract_text(1000753, 0)

Clearly some extractions have failed, about 40% currently which is too high. These will require a case by case handling. Below is a problem for novavax. Also sometimes edgar failes as per error message abover (we could just try catch this case).

## Finding start sequence

In [ ]:
def get_tokens(cik, doc_num):
    company = Company(str(cik), str(cik))
    tree = company.get_all_filings(filing_type = "10-K")
    docs = Company.get_documents(tree, no_of_documents=doc_num+1)
    fs_text = doc.text_content()
    fs_text = fs_text.replace('\t', ' ')
    fs_text = fs_text.replace('\n', ' ')
    return fs_text.split()

In [ ]:
all_tokens = get_tokens(1000697, 0)

In [ ]:
def find_smoothed(tokens, string, smoothing):
    contains = np.array([x.lower().find(string) != -1 for x in tokens])
    return np.minimum(np.convolve(contains, np.ones(smoothing)), 1)

In [ ]:
win = 3
loc_business = find_smoothed(all_tokens, 'business', win)
loc_item = find_smoothed(all_tokens, 'item', win)
loc_1 = find_smoothed(all_tokens, '1', win)

In [ ]:
pp.plot(loc_business+loc_item+loc_1)

In [ ]:
win=3
loc_item = find_smoothed(all_tokens, 'item', win)
loc_1a = find_smoothed(all_tokens, '1a', win)
loc_risk = find_smoothed(all_tokens, 'risk', win)
loc_factors = find_smoothed(all_tokens, 'factors', win)

In [ ]:
pp.plot(loc_item+ loc_1a+loc_risk+loc_factors)

# Edgar Classification

## Manually Classify Data

In [ ]:
my_path = '/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/Data/ClassifiedSections'
os.chdir(my_path)

In [ ]:
tickers = set([x.split('_')[0] for x in os.listdir(my_path)])

loop_over = [(x, y) for x, y in zip(ciks['symbol'].values, ciks['cik'].values)]
for ticker, cik in loop_over:
    if ticker in tickers:
        continue
    company = Company(ticker, str(cik))
    tree = company.get_all_filings(filing_type = "10-K")
    docs = Company.get_documents(tree, no_of_documents=5)
    fs_text = docs[0].text_content().lower()
    
    item_symbol = re.compile('item\s\d.')
    sections = item_symbol.split(fs_text)
    
    found_bd = False
    
    for i, section in enumerate(sections):
        tokenised = re.split(r'[\s\n]+', section)
        if (len(tokenised) > 2000) and (len(tokenised) < 200000):
            if not found_bd:
                decision = input(' '.join(tokenised[:50]))
            if decision == '0' or found_bd:
                # not a business description
                fn = f'{ticker}_section{i}_0.txt'
            else:
                fn = f'{ticker}_section{i}_1.txt'
                found_bd = True
            f = open(fn, 'a')
            f.write(section)
            f.close()

In [ ]:
len(set([x.split('_')[0] for x in os.listdir(my_path)]))

## Train Classifier

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.metrics import plot_confusion_matrix

In [ ]:
n_features = 1000
n_subspace = 30

In [ ]:
file_names = os.listdir(my_path)
y = []
X_raw = []
for fn in file_names:
    fn_first = fn.split('.')[0]
    y.append(int(fn_first.split('_')[2]))
    f = open(fn, 'r')
    f_text = f.read()
    f.close()
    X_raw.append(f_text)

In [ ]:
y = np.array(y)
X_train, X_valid, y_train, y_valid = train_test_split(X_raw, y)

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
tf_vectorizer.fit(X_train)

X_train = tf_vectorizer.transform(X_train)
X_valid = tf_vectorizer.transform(X_valid)

In [ ]:
svd = TruncatedSVD(n_components=n_subspace, n_iter=7, random_state=42)
svd.fit(X_train)

X_train = svd.transform(X_train)
X_valid = svd.transform(X_valid)

In [ ]:
scaler = StandardScaler()
scaler.fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)

In [ ]:
# svc_classifier = SVC(C=1, kernel='rbf', class_weight='balanced')
svc_classifier = SVC(C=1, kernel='rbf', gamma=10, class_weight={0: 1, 1: 2.5})
svc_classifier.fit(X_train, y_train);

In [ ]:
plot_confusion_matrix(svc_classifier, X_train, y_train)

In [ ]:
plot_confusion_matrix(svc_classifier, X_valid, y_valid)

## Problem here is getting classifer to be sparse, but not too sparse!

In [ ]:
svc_classifier.predict(X_train)[:20]

In [ ]:
y_train[:20]

## Automatically Scrape and Classify

## Note could search all documents!!

In [ ]:
my_path = '/home/lawrence/Personal/Masters/COMP0087_ Natural_Language_Processing/Project/Data/AutoClassifiedSections'
os.chdir(my_path)

loop_over = [(x, y) for x, y in zip(ciks['symbol'].values, ciks['cik'].values)]

while 1:
    passed = True
#     try:
    tickers = set([x.split('_')[0] for x in os.listdir(my_path)])
    for ticker, cik in loop_over:
        print(f'Processing {ticker}')
        if ticker in tickers:
            continue
        company = Company(ticker, str(cik))
        tree = company.get_all_filings(filing_type = "10-K")
        docs = Company.get_documents(tree, no_of_documents=5)
        fs_text = docs[0].text_content().lower()

        item_symbol = re.compile('item\s\d.')
        sections = item_symbol.split(fs_text)
        lengths = [len(re.split(r'[\s\n]+', x)) for x in sections]
        sections = [x for x, y in zip(sections, lengths) if (y > 2000) and (y < 200000)]
        print(f'Found {len(sections)} sections')

        X_classify = tf_vectorizer.transform(sections)
        X_classify = svd.transform(X_classify)
        X_classify = scaler.transform(X_classify)
        y_hat = svc_classifier.predict(X_classify)

        for i, section in enumerate(sections):
            if y_hat[i] == 0:
                # not a business description
                fn = f'{ticker}_section{i}_0.txt'
            else:
                fn = f'{ticker}_section{i}_1.txt'
            f = open(fn, 'a')
            f.write(section)
            f.close()
                
#     except:
#         passed = False
        
    if passed:
        break

In [ ]:
svc_classifier.predict(X_classify)

In [ ]:
sections

# LDA Companies

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx

import os
os.chdir('/Users/brandon/Documents/UCL/COMP0087/CW2')

## (Hyper)parameters

In [ ]:
n_features = 1000
n_components = 10
n_top_words = 20

## Load Data

In [ ]:
f = open("bds_1.txt", "r")
f_lines = f.readlines()
f.close()

In [ ]:
company_ids = f_lines[0::2]
company_descriptions = f_lines[1::2]

In [ ]:
len_descriptions = [len(x) for x in company_descriptions]
plt.figure(figsize=[10, 5])
plt.subplot(1, 2, 1)
plt.hist(len_descriptions, bins=100);
plt.title('Description Lengths');

plt.subplot(1, 2, 2)
plt.hist([len(x) for x in company_ids], bins=100);
plt.title('Id Lengths');

In [ ]:
valid_descriptions = np.array(len_descriptions) > 3000
n_samples = np.sum(valid_descriptions)
n_samples

In [ ]:
company_ids = np.array(company_ids)[valid_descriptions]
company_descriptions = [x for x in company_descriptions if len(x) > 3000]

In [ ]:
some_lengths = [len(x) for x in company_descriptions if len(x) < 5e5]
plt.hist(some_lengths, bins=100);

## Feature Extraction via Term Frequency
This handles special characters, takes lowercase and removes common words.

In [ ]:
tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2,
                                max_features=n_features,
                                stop_words='english')
tf = tf_vectorizer.fit_transform(company_descriptions)

In [ ]:
tf.shape

## Fitting LDA
Online LDA using VB. Really fast!
If there are W words in dictionary, D documents in corpus and K topics then it learns:
 - a matrix K $\times$ W of word probabilities per topic that allow you to determine what the topics mean
 - a matrix D $\times$ K of topic probabilities per document that allows you to cluster documents

In [ ]:
lda = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)

In [ ]:
lda.fit(tf)

In [ ]:
lda.perplexity(tf)

## What do the Topics Mean?

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title):
    fig, axes = plt.subplots(2, 5, figsize=(15, 15), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=40)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
# names of words 
tf_feature_names = tf_vectorizer.get_feature_names()
plot_top_words(lda, tf_feature_names, n_top_words, 'Topics in LDA model')

## Assigining Topics to Companies

In [ ]:
# Produce a companies by topics matrix of prob of each topic by company
topics_companies = lda.transform(tf)

In [ ]:
primary_topics = np.argmax(topics_companies, axis=1)

Picking out companies in topic 1 i.e. pharma

In [ ]:
topic_id = 1
topic_locs = np.argwhere(primary_topics==topic_id)

this_ind = 5
print(company_ids[topic_locs[this_ind]])

In [ ]:
print(company_descriptions[int(topic_locs[this_ind])])

Now topic 9 oil & gas

In [ ]:
topic_id = 9
topic_locs = np.argwhere(primary_topics==topic_id)

this_ind = 5
print(company_ids[topic_locs[this_ind]])

In [ ]:
print(company_descriptions[int(topic_locs[this_ind])])

Now topic 6 ... REITS. Note how high up COVID is in this description and consider its impact on companies that lease office or retail space.

In [ ]:
topic_id = 6
topic_locs = np.argwhere(primary_topics==topic_id)

this_ind = 5
print(company_ids[topic_locs[this_ind]])

In [ ]:
print(company_descriptions[int(topic_locs[this_ind])])

In [ ]:
this_ind = 10
# prison REIT lol
print(company_descriptions[int(topic_locs[this_ind])])

# Visualising Clusters

In [ ]:
val, vec = np.linalg.eig(topics_companies.T @ topics_companies)
projection_01 = topics_companies @ vec[:, 0:2]
projection_12 = topics_companies @ vec[:, 1:3]
projection_23 = topics_companies @ vec[:, 2:4]
projection_34 = topics_companies @ vec[:, 3:5]

In [ ]:
plt.figure(figsize=[15, 15])
plt.subplot(2, 2, 1)
plt.scatter(projection_01[primary_topics==1, :][:, 0], projection_01[primary_topics==1, :][:, 1])
plt.scatter(projection_01[primary_topics==4, :][:, 0], projection_01[primary_topics==4, :][:, 1])
plt.scatter(projection_01[primary_topics==6, :][:, 0], projection_01[primary_topics==6, :][:, 1])
plt.scatter(projection_01[primary_topics==9, :][:, 0], projection_01[primary_topics==9, :][:, 1])

plt.subplot(2, 2, 2)
plt.scatter(projection_12[primary_topics==1, :][:, 0], projection_12[primary_topics==1, :][:, 1])
plt.scatter(projection_12[primary_topics==4, :][:, 0], projection_12[primary_topics==4, :][:, 1])
plt.scatter(projection_12[primary_topics==6, :][:, 0], projection_12[primary_topics==6, :][:, 1])
plt.scatter(projection_12[primary_topics==9, :][:, 0], projection_12[primary_topics==9, :][:, 1])
plt.legend(['Pharma', 'Banks', 'REITs', 'Energy'])

plt.subplot(2, 2, 3)
plt.scatter(projection_23[primary_topics==1, :][:, 0], projection_23[primary_topics==1, :][:, 1])
plt.scatter(projection_23[primary_topics==4, :][:, 0], projection_23[primary_topics==4, :][:, 1])
plt.scatter(projection_23[primary_topics==6, :][:, 0], projection_23[primary_topics==6, :][:, 1])
plt.scatter(projection_23[primary_topics==9, :][:, 0], projection_23[primary_topics==9, :][:, 1])

plt.subplot(2, 2, 4)
plt.scatter(projection_34[primary_topics==1, :][:, 0], projection_34[primary_topics==1, :][:, 1])
plt.scatter(projection_34[primary_topics==4, :][:, 0], projection_34[primary_topics==4, :][:, 1])
plt.scatter(projection_34[primary_topics==6, :][:, 0], projection_34[primary_topics==6, :][:, 1])
plt.scatter(projection_34[primary_topics==9, :][:, 0], projection_34[primary_topics==9, :][:, 1])

## Visualising the Network

In [ ]:
sub_select = np.zeros(len(topics_companies), dtype=bool)
sub_select[:50] = True

In [ ]:
adj_matrix = topics_companies[sub_select, :] @ topics_companies[sub_select, :].T

In [ ]:
np.fill_diagonal(adj_matrix, 1)
am_cut = 0.2
adj_matrix[adj_matrix < am_cut] = 0
adj_matrix[adj_matrix >= am_cut] = 1

In [ ]:
G = nx.from_numpy_matrix(adj_matrix)
these_names = company_ids[sub_select]
node_mapping = {i: x for i, x in enumerate(these_names)}
G = nx.relabel_nodes(G, node_mapping)

In [ ]:
plt.figure(figsize=[15, 15])
nx.draw(G, with_labels=True)

# LDA Selecting K

## Load Data

In [ ]:
f = open("bds_1.txt", "r")
f_lines = f.readlines()
f.close()

company_ids = f_lines[0::2]
company_descriptions = f_lines[1::2]

len_descriptions = [len(x) for x in company_descriptions]
valid_descriptions = np.array(len_descriptions) > 3000
company_ids = np.array(company_ids)[valid_descriptions]

company_descriptions = [x for x in company_descriptions if len(x) > 3000]

## Process Data

### Feature Extraction

In [ ]:
n_features = 4000
tf_vectorizer = CountVectorizer(max_features=n_features, max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(company_descriptions)

## Fit LDA

In [ ]:
n_components = 10
lda_10 = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_10.fit(tf)

In [ ]:
n_components = 15
lda_15 = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_15.fit(tf)

In [ ]:
n_components = 20
lda_20 = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_20.fit(tf)

## Coherence Measure
See http://qpleple.com/topic-coherence-to-evaluate-topic-models/

In [ ]:
Dwi = np.array(np.sum(tf > 0, axis=0))[0]

W_bin = tf
W_bin[W_bin > 0] = 1
Dwi_wj = W_bin.T @ W_bin

score_umass = np.log((Dwi_wj.toarray() + 1)/ Dwi)

In [ ]:
def topic_coherence(model, n_top_words, pair_score):
    coherences = []
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        coh = 0
        for i in range(len(top_features_ind)):
            for j in range(i):
                coh += pair_score[top_features_ind[i], top_features_ind[j]]
        coherences.append(coh)
    return coherences

## Check Coherence for Different K

In [ ]:
k_values = [2, 3, 4, 5, 6, 7, 8, 9, 10, 15, 20, 25, 30, 35, 40, 45, 50]
tc_values = []
runs = 20
for k in k_values:
    print(f'running k = {k}')
    this_c = []
    for run in range(runs):
    
        lda = LatentDirichletAllocation(n_components=k, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.,
                                    random_state=0)
        lda.fit(tf)
        this_c.append(np.median(topic_coherence(lda, 10, score_umass)))
    tc_values.append(this_c)

In [ ]:
plt.plot(k_values, tc_values)

## Plotting Top Words

In [ ]:
def plot_top_words(model, feature_names, n_top_words, title):
    K = len(model.components_)
    n_x = 5
    n_y = int(np.ceil(K / n_x))
    fig, axes = plt.subplots(n_y, n_x, figsize=(3 * n_x, 7.5 * n_y), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(model.components_):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 30})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=20)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=30)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
tf_feature_names = tf_vectorizer.get_feature_names()
plot_top_words(lda_10, tf_feature_names, 10, 'Topics in LDA 10 model')

In [ ]:
plot_top_words(lda_15, tf_feature_names, 10, 'Topics in LDA 15 model')

In [ ]:
plot_top_words(lda_20, tf_feature_names, 10, 'Topics in LDA 20 model')

# LDA Stock Returns

In [ ]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as pp
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

## Business Description Data
### SP500 Data

In [ ]:
my_path = Path('/Users/brandon/Documents/UCL/COMP0087/CW2/SP500')
os.chdir(my_path)
fns = os.listdir(my_path)

ticker_sp50 = []
sector_sp50 = []
bds_sp50 = []
for fn in fns:
    fn_first = fn.split('.txt')[0]
    ticker = fn_first.split('_')[0]
    sector = fn_first.split('_')[1]
    f = open(fn, 'r', encoding="utf8")
    f_text = f.read()
    f.close()
    
    ticker_sp50.append(ticker)
    sector_sp50.append(sector)
    bds_sp50.append(f_text)

## Larger Dataset - excluding SP500

In [ ]:
os.chdir(Path('/Users/brandon/Documents/UCL/COMP0087/CW2'))
f = open("bds_1.txt", "r", encoding="utf8")
f_lines = f.readlines()
f.close()

company_ids_all = f_lines[0::2]
company_descriptions_all = f_lines[1::2]
company_tickers = [x.split(':')[0] for x in company_ids_all]
removeSP = np.in1d(np.array(company_tickers), list(ticker_sp50))

bds_all = []
ticker_all = []
for i, d in enumerate(company_descriptions_all):
    if (len(d) > 3000) and not removeSP[i]:
        bds_all.append(d)
        ticker_all.append(company_ids_all[i])

## Get Returns Data

In [ ]:
start_d = np.datetime64('2018-01-01')
end_d = np.datetime64('2020-01-01')
business_ds = pd.date_range(start_d, end_d, freq='B')

my_path = Path('/Users/brandon/Documents/UCL/COMP0087/CW2')
os.chdir(my_path)
price_data = pd.read_csv('Price.csv')

select_these = np.in1d(price_data.tic.values, list(ticker_sp50))
price_sp50 = price_data.loc[select_these, ['tic', 'datadate', 'prccd']]
price_sp50['datadate'] = pd.to_datetime(price_sp50['datadate'], format='%Y%m%d')
price_sp50 = pd.pivot_table(price_sp50,index='datadate',columns='tic',values='prccd')
price_sp50 = price_sp50.ffill(limit=5)
price_sp50 = price_sp50.reindex(business_ds)
price_sp50 = price_sp50.dropna(axis=0)

returns_sp50 = np.log(price_sp50) - np.log(price_sp50.shift(1))
returns_sp50 = returns_sp50.dropna(axis=0)

## Explaining Returns with LDA
### Train LDA on Larger Dataset

In [ ]:
n_features = 4000
tf_vectorizer = CountVectorizer(max_features=n_features, max_df=0.95, min_df=2, stop_words='english')
tf = tf_vectorizer.fit_transform(bds_all)

n_components = 20
lda_20 = LatentDirichletAllocation(n_components=n_components, max_iter=5,
                                learning_method='online',
                                learning_offset=50.,
                                random_state=0)
lda_20.fit(tf)

### Get Features for S&P 500

In [ ]:
tf_sp50 = tf_vectorizer.transform(bds_sp50)

features_sp50 = lda_20.transform(tf_sp50)

features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)

### Loop Over Dates and Perform OLS Regression

In [ ]:
all_dates = returns_sp50.index
max_f = 19
adj_r2_features = []

for dd in all_dates:
    reg_data = returns_sp50.loc[[dd]].transpose().join(features_sp50_df.loc[:, 0:max_f]).dropna(axis=0).values
    y = reg_data[:, 0]
    X = reg_data[:, 1:]

    std_scaler = StandardScaler()
    X = std_scaler.fit_transform(X)

    X = sm.add_constant(X, prepend=False)
    ols_model = sm.OLS(y, X)
    res = ols_model.fit()
    adj_r2_features.append(res.rsquared_adj)

In [ ]:
pp.plot(all_dates, adj_r2_features)

### Comparing Different K

Next, calculate average regression adjusted $R^2$ for different K

In [ ]:
tf_sp50 = tf_vectorizer.transform(bds_sp50)
all_ks = [5, 10, 15, 20, 25, 30, 40, 50]
num_trials = 20
all_adj_r2 = []
for k in all_ks:
    print(f'Running for k = {k}')
    adj_r2_k = []
    for t in range(num_trials):
        this_lda = LatentDirichletAllocation(n_components=k, max_iter=5,
                                    learning_method='online',
                                    learning_offset=50.)
        this_lda.fit(tf)
    
        features_sp50 = this_lda.transform(tf_sp50)
        features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)
        adj_r2_dates = []
        for dd in all_dates:
            reg_data = returns_sp50.loc[[dd]].transpose().join(features_sp50_df.loc[:, 0:max_f]).dropna(axis=0).values
            y = reg_data[:, 0]
            X = reg_data[:, 1:]

            std_scaler = StandardScaler()
            X = std_scaler.fit_transform(X)

            X = sm.add_constant(X, prepend=False)
            ols_model = sm.OLS(y, X)
            res = ols_model.fit()
            adj_r2_dates.append(res.rsquared_adj)
        adj_r2_k.append(np.mean(adj_r2_dates))
    all_adj_r2.append(adj_r2_k)

In [ ]:
[np.mean(x) for x in all_adj_r2]

In [ ]:
[np.std(x) for x in all_adj_r2]

# KMeans Stock Returns

In [ ]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as pp
from pathlib import Path

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [ ]:
import pandas as pd
import nltk as nltk
import string

from time import time
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.stem import SnowballStemmer

from sklearn.cluster import KMeans 
from sklearn.cluster import AgglomerativeClustering

## KMeans on S&P500

In [ ]:
#Instance of Lemmatizer
lemmatizer = WordNetLemmatizer()

### Pre-processing larger dataset

In [ ]:
#preprocess larger dataset
t0 = time()
#Remove puncuation
bds_all2 = bds_all[:]
for i in range(len(bds_all)):
    for k in string.punctuation:
        bds_all2[i] = bds_all2[i].replace(k, '')

#Tokenise and remove stopwords
tokenised = []
for i in range(len(bds_all2)):
    text = bds_all2[i].split()
    text = [word.lower() for word in text if not word.lower() in stopwords.words('english')]
    tokenised.append(text)

#Lemmatizes each word
lemmad_text = []
for i in range(len(tokenised)): #CHANGE THIS
# for i in range(1):
    print(i)
    lemmad_array = tokenised[i]
    lemmad = [lemmatizer.lemmatize(word.lower()) for word in lemmad_array]
    lemmad = ' '.join(lemmad)
    lemmad_text.append(lemmad)

print("done in %fs" % (time() - t0))
print(lemmad_text[0])


### Pre-processing S&P500

In [ ]:
#preprocess sp500
t0 = time()
#Remove puncuation
bds_sp50_2 = bds_sp50[:]
for i in range(len(bds_sp50_2)):
    for k in string.punctuation:
        bds_sp50_2[i] = bds_sp50_2[i].replace(k, '')

#Tokenise and remove stopwords
tokenised_sp50 = []
for i in range(len(bds_sp50_2)):
    text = bds_sp50_2[i].split()
    text = [word.lower() for word in text if not word.lower() in stopwords.words('english')]
    tokenised_sp50.append(text)

#Lemmatizes each word
lemmad_text_sp50 = []
for i in range(len(tokenised_sp50)): #CHANGE THIS
# for i in range(1):
    print(i)
    lemmad_array = tokenised_sp50[i]
    lemmad = [lemmatizer.lemmatize(word.lower()) for word in lemmad_array]
    lemmad = ' '.join(lemmad)
    lemmad_text_sp50.append(lemmad)

print("done in %fs" % (time() - t0))
print(lemmad_text_sp50[0])


### Get Features

In [ ]:
cv_lem = CountVectorizer()
X_lemmad = cv_lem.fit_transform(lemmad_text)
print(X_lemmad.shape)

### Train Kmeans using elbow method to find best k based on inertia

In [ ]:
t0 = time()
#Training Kmeans
wcss_lemmad = [] #CHANGE
for i in range(1, 20): #max of 20 clusters otherwise might take too long
    kmeans_lem = KMeans(n_clusters = i, init='k-means++', max_iter = 300, n_init = 10, random_state = 0, verbose = True)
    kmeans_lem.fit(X_lemmad) #CHANGE
    wcss_lemmad.append(kmeans_lem.inertia_) #CHANGE
    
print("done in %fs" % (time() - t0))
    
#Visualisation of inertia (how far away the points within a cluster are, want small inertia)
pp.plot(range(1,20),wcss_lemmad[0:20]) #CHANGE
pp.title('the elbow method')
pp.xlabel('no of clusters')
pp.ylabel('wcss')
pp.show

### Using best k from elbow method on larger dataset

In [ ]:
#applying K-means based on above elbow method, can change true_k to the desired number of clusters
true_k = 10
model_true = KMeans(n_clusters=true_k, init='k-means++', n_init=1)
model_true.fit(X_lemmad) #CHANGE

#Print results
print("top terms for cluster:")
order_centroids_lem = model_true.cluster_centers_.argsort()[:, ::-1]
terms_lem = cv_lem.get_feature_names()
for i in range(true_k):
    print("cluster %d:" % i),
    for ind in order_centroids_lem[i, :10]: #selects top terms
        print(' %s' % terms_lem[ind]),
    print
    
print("\n")

## Explaining Returns with Kmeans

### Get Features for S&P500

In [ ]:
tf_sp50 = cv_lem.transform(lemmad_text_sp50)

features_sp50 = model_true.transform(tf_sp50)

features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)

### Loop Over Dates and Perform OLS Regression

In [ ]:
all_dates = returns_sp50.index
max_f = 19
adj_r2_features = []

for dd in all_dates:
    reg_data = returns_sp50.loc[[dd]].transpose().join(features_sp50_df.loc[:, 0:max_f]).dropna(axis=0).values
    y = reg_data[:, 0]
    X = reg_data[:, 1:]

    std_scaler = StandardScaler()
    X = std_scaler.fit_transform(X)

    X = sm.add_constant(X, prepend=False)
    ols_model = sm.OLS(y, X)
    res = ols_model.fit()
    adj_r2_features.append(res.rsquared_adj)

In [ ]:
pp.plot(all_dates, adj_r2_features)

In [ ]:
tf_sp50 = cv_lem.transform(lemmad_text_sp50)
all_ks = [5, 10, 15, 20, 25, 30, 40, 50]
num_trials = 20
all_adj_r2 = []
for k in all_ks:
    print(f'Running for k = {k}')
    adj_r2_k = []
    for t in range(num_trials):
        
        this_kmeans = KMeans(n_clusters = k, init='k-means++', n_init = 1, random_state = 0, verbose = True)
        this_kmeans.fit(X_lemmad)
        
        features_sp50 = this_kmeans.transform(tf_sp50)
        features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)
        
#         this_lda = LatentDirichletAllocation(n_components=k, max_iter=5,
#                                     learning_method='online',
#                                     learning_offset=50.)
#         this_lda.fit(tf)
    
#         features_sp50 = this_lda.transform(tf_sp50)
#         features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)
        adj_r2_dates = []
        for dd in all_dates:
            reg_data = returns_sp50.loc[[dd]].transpose().join(features_sp50_df.loc[:, 0:max_f]).dropna(axis=0).values
            y = reg_data[:, 0]
            X = reg_data[:, 1:]

            std_scaler = StandardScaler()
            X = std_scaler.fit_transform(X)

            X = sm.add_constant(X, prepend=False)
            ols_model = sm.OLS(y, X)
            res = ols_model.fit()
            adj_r2_dates.append(res.rsquared_adj)
        adj_r2_k.append(np.mean(adj_r2_dates))
    all_adj_r2.append(adj_r2_k)

In [ ]:
[np.mean(x) for x in all_adj_r2]

In [ ]:
[np.std(x) for x in all_adj_r2]

# NVDM Python Scripts

In [ ]:
"""
Command line version of notebook.
"""
#!/usr/bin/env python


import argparse
import pickle
from timeit import default_timer as timer
from datetime import timedelta

from collections import Counter
from torchtext.vocab import Vocab
import torch
from torch.utils.data import Dataset, DataLoader

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

import data_handling as data
from models import nvdm

parser = argparse.ArgumentParser()
parser.add_argument("-k", type=int, dest="num_topics", required=True)
parser.add_argument("-e", type=int, dest="num_epochs", required=True)
args = parser.parse_args()
print(args)

print("Load data..")

# Data that has already been preprocessed
# Generated by applying pp.preprocess_text() to each BD,
# then saved to a TSV
DATA_CLEAN_PATH = "./data/bds_1_clean.txt"
IDs_raw, BDs_raw = data.load_raw(DATA_CLEAN_PATH)

# Some entries have empty BDs, so filter those out
IDs = []
BDs = []
for iid, bd in zip(IDs_raw, BDs_raw):
    if len(bd) > 0:
        IDs.append(iid)
        BDs.append(bd)

print(len(IDs), len(BDs))

# Build frequency table
# (cleaned data joins tokens by space)
counter = Counter()
for desc in BDs:
    counter.update(desc.split(" "))

# PyTorch torchtext vocabulary converts tokens to indices and vice versa.
# Also has an '<unk>' for OOV words (might be useful later).
# vocab = Vocab(counter,
#               max_size=10000,
#               min_freq=1,
#               specials=['<unk>'])
# print(len(vocab))
# Load the same precomputed vocabulary each time for consistency.
with open("./vocabs/vocab_bds_1_clean_10000.pickle", "rb") as f:
    vocab = pickle.load(f)


class BDDataset(Dataset):
    """ Very simple dataset object. Stores all the passages.
    
    This is just for compatibility with PyTorch DataLoader.
    """
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]


# "Preprocessing" function: just splits the text
# The file's text is already preprocessed.
def text_pipeline(text):
    return [vocab[token] for token in text.split(" ")]


def collate_batch(batch):
    """ Convert a batch of text (each a list of tokens) into appropriate torch tensors.
    
    Modification of https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html.
    We don't need labels.
    """
    # Offsets tells the model (which will use EmbeddingBag) where each text starts.
    text_list, offsets = [], [0]
    for _text in batch:
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), offsets.to(device)

# Create data loader to iterate over dataset in batches during training/evaluation
dataset = BDDataset(BDs)
batch_size = 64
data_loader = DataLoader(dataset,
                         batch_size=batch_size,
                         shuffle=True,
                         collate_fn=collate_batch)
hidden_size = 500
num_topics = args.num_topics

# Training setup

# Total number of epochs
outer_epochs = args.num_epochs

model = nvdm.NVDM(len(vocab), hidden_size, num_topics, 1, device)
model = model.to(device)
model.train()

# Trains both the encoder and decoder at the same time.
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

start_time = timer()
print("Start training...")


for epoch in range(outer_epochs):

    loss_sum = 0.0
    rec_sum = 0.0
    kl_sum = 0.0
    n = len(data_loader)

    for idx, (text, offsets) in enumerate(data_loader):
        text = text.to(device)
        offsets = offsets.to(device)

        optimizer.zero_grad()
        loss_dict = model(text, offsets, kl_weight=1.0)
        loss = loss_dict["total"].sum()
        loss.backward()

        optimizer.step()

        # For printing
        loss_sum += loss.item()
        rec_sum += loss_dict["rec"].sum().item()
        kl_sum += loss_dict["kl"].sum().item()

    print(f"[Time: {timedelta(seconds=timer() - start_time)}, Epoch {epoch + 1}] Loss {loss_sum/n}, Rec {rec_sum/n}, KL {kl_sum/n}")

MODELSAVE_PATH = f"./modelsaves/nvdm_k{num_topics}_{outer_epochs}epochs.pt"
print(f"Save model... to {MODELSAVE_PATH}")
torch.save(model.state_dict(), MODELSAVE_PATH)
print("Done.")


In [ ]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

PORTER_STEMMER = PorterStemmer()
STOPWORDS = set(stopwords.words('english'))


def preprocess_text(text):
    """ Preprocess one line of text. """
    # NLTK also separates out punctuation, but keeps them in the list.
    # e.g. ',' is a token.
    tokens = nltk.word_tokenize(text, language="english")

    tokens_new = []
    for token in tokens:
        if token in STOPWORDS:
            continue

        if not token.isalnum():
            # Only keep words containing alphanumeric characters
            # Tokens containing only punctuation, symbols etc. are removed
            continue

        # TODO: do further preprocessing if necessary,
        #       (if vocab size is too large) e.g. handle Unicode, lowercase

        # Stem word
        # TODO: map stemmed words to their original to recover them???
        token_new = PORTER_STEMMER.stem(token)
        tokens_new.append(token_new)

    return tokens_new


In [ ]:
"""
Load text files.
"""
import csv


def load_raw(path):
    """ Load data in the same format as bds_1.txt. """
    with open(path) as f:
        lines = f.read().split("\n")

    ids = []
    bds = []
    for i, p in enumerate(lines):
        if i % 2 == 0:
            ids.append(p)
        else:
            bds.append(p)

    # Original bds_1.txt has a newline at the end of file,
    # so remove this blank line
    if len(lines) % 2 != 0 and ids[-1] == '':
        ids.pop(-1)

    return ids, bds


def write_data(ids, bds, path):
    """ Write some ID/BD pairs to a tab-separate file. """
    # Python docs says newline="" must be specified for csv
    assert len(ids) == len(bds)

    with open(path, "w", newline="") as outfile:
        for iid, bd in zip(ids, bds):
            outfile.write(iid + "\n")
            outfile.write(" ".join(bd) + "\n")


# NVDM Experiments

In [ ]:
# Standard library imports
from timeit import default_timer as timer
from datetime import timedelta
import pickle
from collections import Counter

In [ ]:
# External libraries
import numpy as np
import matplotlib.pyplot as plt
import tqdm

In [ ]:
# PyTorch stuff
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import Vocab
import torch
from torch.utils.data import Dataset, DataLoader
from torch import nn
import torch.nn.functional as F

# Autoselect target device.
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

In [ ]:
%load_ext autoreload
%autoreload 2

# Custom modules.

import data_handling as data
import preprocess as pp
from models import nvdm

In [ ]:
# # Original data
# DATA_RAW_PATH = "./data/bds_1.txt"
# IDs, BDs = data.load_raw(DATA_RAW_PATH)

In [ ]:
# Data that has already been preprocessed, same format as original bds_1.txt.
# Generated by applying pp.preprocess_text() to each BD and saving this to a text file.
DATA_CLEAN_PATH = "./data/bds_1_clean.txt"
IDs_raw, BDs_raw = data.load_raw(DATA_CLEAN_PATH)

In [ ]:
# Some entries have empty BDs, so filter those out
IDs = []
BDs = []
for iid, bd in zip(IDs_raw, BDs_raw):
    if len(bd) > 0:
        IDs.append(iid)
        BDs.append(bd)

print(len(IDs), len(BDs))

In [ ]:
# This cell shows how the torchtext Vocab was created.
# The object is saved to disk, so this cell only needs to uncommented and run once.
# Build frequency table
# (cleaned data joins tokens by space)
# counter = Counter()
# for desc in BDs:
#     counter.update(desc.split(" "))

# PyTorch torchtext vocabulary converts tokens to indices and vice versa.
# Also has an '<unk>' for OOV words (might be useful later).
# vocab = Vocab(counter,
#               max_size=10000,
#               min_freq=1,
#               specials=['<unk>'])
# print(len(vocab))
# actual is 70770 without max_size restriction

# Save the vocab to file
# with open("./vocabs/vocab_bds_1_clean_10000.pickle", "wb") as f:
#     pickle.dump(vocab, f)

In [ ]:
# If the vocab was already created, load the vocab from file.
with open("./vocabs/vocab_bds_1_clean_10000.pickle", "rb") as f:
    vocab = pickle.load(f)

print(len(vocab))

In [ ]:
class BDDataset(Dataset):
    """ Very simple dataset object. Stores all the passages.
    
    This is just for compatibility with PyTorch DataLoader.
    """
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]

In [ ]:
# "Preprocessing" function: just splits the text
# The file's text is already preprocessed.
def text_pipeline(text):
    return [vocab[token] for token in text.split(" ")]

def collate_batch(batch):
    """ Convert a batch of text (each a list of tokens) into appropriate torch tensors.
    
    Modification of https://pytorch.org/tutorials/beginner/text_sentiment_ngrams_tutorial.html.
    We don't need labels.
    """
    # Offsets tells the model (which will use EmbeddingBag) where each text starts.
    text_list, offsets = [], [0]
    for _text in batch:
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text)
        offsets.append(processed_text.size(0))

    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return text_list.to(device), offsets.to(device)

In [ ]:
# Create data loader to iterate over dataset in batches during training/evaluation
dataset = BDDataset(BDs)
batch_size = 64
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_batch)
hidden_size = 500
num_topics = 10

In [ ]:
# Training setup
# Better to run run_nvdm.py in the command line to train than here.
# Uncomment the code below to train if really needed.

# # Total number of epochs
# outer_epochs = 200

# # Epochs for training the encoder/decoder on each alternation.
# inner_epochs = 1

# model = nvdm.NVDM(len(vocab), hidden_size, num_topics, 1, device)
# model = model.to(device)
# model.train()

# # Trains both the encoder and decoder at the same time.
# # The original paper alternates between the encoder and decoder,
# # but this is very unstable and hard to converge, from observation.
# optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# start_time = timer()

# for epoch in range(outer_epochs):

#     loss_sum = 0.0
#     rec_sum = 0.0
#     kl_sum = 0.0
#     n = len(data_loader)

#     for idx, (text, offsets) in enumerate(data_loader):
#         text = text.to(device)
#         offsets = offsets.to(device)

#         optimizer.zero_grad()
#         loss_dict = model(text, offsets, kl_weight=1.0)
#         loss = loss_dict["total"].sum()
#         loss.backward()

#         optimizer.step()

#         # For printing
#         loss_sum += loss.item()
#         rec_sum += loss_dict["rec"].sum().item()
#         kl_sum += loss_dict["kl"].sum().item()

#     model_str = "All" # "Enc" if switch == 0 else "Dec"
#     print(f"[Time: {timedelta(seconds=timer() - start_time)}, Epoch {epoch + 1}] Loss {loss_sum/n}, Rec {rec_sum/n}, KL {kl_sum/n}")

In [ ]:
# Load/save model.

# MODELSAVE_PATH = "./modelsaves/nvdm_from_notebook.pt"
MODELSAVE_PATH = "./modelsaves/nvdm_k10_300epochs.pt"

# Save the trained model to file if needed.
# torch.save(model.state_dict(), MODELSAVE_PATH)

# Otherwise load the model from file if it exists.
# Customize the n_topic (number of topics) according to the file name.
# e.g. "nvdm_k10_300epochs.pt" means n_topic=k=10.
model = nvdm.NVDM(len(vocab), 500, 10, 1, device)
model.load_state_dict(torch.load(MODELSAVE_PATH))
model.eval()

In [ ]:
# Extract the vocab-topic matrix (known as R in the paper).
# It has dimensions |V| x K: vocab size x number of topics
decoder = model.decoder[0]
weights = decoder.weight.data.detach().clone()
weights.size()

In [ ]:
# Look at some words
# manual KNN
from nltk.stem import PorterStemmer
PORTER_STEMMER = PorterStemmer()

# Set of words used in the original paper
candidates = ["weapons", "medical", "companies", "define", "israel", "book"]

for candidate in candidates:
    test_word = PORTER_STEMMER.stem(candidate)
    idx = vocab.stoi[test_word]
    print(test_word, idx)

    # Show top 10 most similar (based on cosine distance)
    sims = F.cosine_similarity(weights[idx].unsqueeze(0), weights)
    sim_vals, sim_idxs = torch.topk(sims, 15)

    # Show ith nearest word and its score.
    for i, v in zip(sim_idxs, sim_vals):
        print(f"{vocab.itos[i]}\t{v.item()}")
    
    print("-----------")

In [ ]:
# Look at most similar words per topic vector.

V, K = weights.size()
for i in range(K):
    print(f"Topic {i+1}")
    vals, idxs = torch.topk(torch.abs(weights[:, i]), 30)
    for i, v in zip(idxs, vals):
        print(f"{vocab.itos[i]}\t{v.item()}")
    print("------------")

## Part 2 Full Evaluation

In [ ]:
def analysis(model):
    """ Qualitative analysis of topic model. """
    
    PORTER_STEMMER = PorterStemmer()
    # Set of words used in the original NVDM paper
    candidates = ["weapons", "medical", "companies", "define", "israel", "book"]
    
    # Extract the vocab-topic matrix (known as R in the paper).
    # It has dimensions |V| x K: vocab size x number of topics
    decoder = model.decoder[0]
    weights = decoder.weight.data.detach().clone()

    for candidate in candidates:
        test_word = PORTER_STEMMER.stem(candidate)
        idx = vocab.stoi[test_word]
        print(test_word, idx)

        # Show top 10 most similar (based on cosine distance)
        sims = F.cosine_similarity(weights[idx].unsqueeze(0), weights)
        sim_vals, sim_idxs = torch.topk(sims, 15)

        # Show ith nearest word and its score.
        for i, v in zip(sim_idxs, sim_vals):
            print(f"{vocab.itos[i]}\t{v.item()}")

        print("-----------")
    
    V, K = weights.size()
    for i in range(K):
        print(f"Topic {i+1}")
        vals, idxs = torch.topk(torch.abs(weights[:, i]), 30)
        for i, v in zip(idxs, vals):
            print(f"{vocab.itos[i]}\t{v.item()}")
        print("------------")

In [ ]:
# Topic coherence.
def umass_score(tf):
    """ Compute topic coherence using UMass metric.
    
    Ref: http://qpleple.com/topic-coherence-to-evaluate-topic-models/
    
    tf: term-frequency matrix for each document.
        Each i^th row is the BOW representation of the i^th document.
    """
    
    # D(wi): count of documents containing the word wi (i.e. df)
    Dwi = np.array(np.sum(tf > 0, axis=0))[0]

    W_bin = np.zeros_like(tf)
    W_bin[tf > 0] = 1
    
    # D(wi, wj): count of documents containing both words wi and wj
    Dwi_wj = W_bin.T @ W_bin

    score_umass = np.log((Dwi_wj + 1)/ Dwi)
    
    return score_umass


def topic_coherence(topic_vocab, n_top_words, pair_score):
    """
    topic_vocab: dimensions (number of topics, vocabulary size).
    model.components_ for LDA, and the "semantic embedding" matrix in the decoder for NVDM.
    
    pair_score: matrix of scores (e.g. UMass)
    """
    coherences = []
    for topic_idx, topic in enumerate(topic_vocab):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        coh = 0
        for i in range(len(top_features_ind)):
            for j in range(i):
                coh += pair_score[top_features_ind[i], top_features_ind[j]]
        coherences.append(coh)
    return coherences

In [ ]:
import os
import re

from nltk.stem import PorterStemmer


MODELSAVE_PATH = "./modelsaves"

models_k = dict()
k_values = []

for filename in os.listdir(MODELSAVE_PATH):
    
    num_topics = filename.split("_")[1][1:]
    num_topics = int(num_topics)
    k_values.append(num_topics)
    
    model = nvdm.NVDM(len(vocab), hidden_size, num_topics, 1, "cpu")
    model.load_state_dict(torch.load(os.path.join(MODELSAVE_PATH, filename), map_location="cpu"))
    model.eval()
    models_k[num_topics] = model


In [ ]:
# Make the BOW matrix manually, using the existing Vocab's token-to-index mapping.
bow_mat = np.zeros((len(BDs), len(vocab)))
for d, bd in enumerate(BDs):
    token_idxs = vocab.lookup_indices(bd.split(" "))
    word_counts = Counter(token_idxs)
    for w, count in word_counts.items():
        bow_mat[d, w] = count   
bow_mat.shape

In [ ]:
score_umass_mat = umass_score(bow_mat)

In [ ]:
k_values.sort()
tc_values = []

for k in k_values:
    print(f'Running k = {k}')
    this_model = models_k[k]
    
    # Extract the topic vocab matrix
    decoder = this_model.decoder[0]
    weights = decoder.weight.data.detach().clone().numpy()
    topic_vocab_mat = weights.T
    coherences = topic_coherence(topic_vocab_mat, 10, score_umass_mat)
    this_c = np.median(coherences)

    tc_values.append(this_c)

In [ ]:
plt.plot(k_values, tc_values);

In [ ]:
def plot_top_words(topic_vocab, feature_names, n_top_words, title):
    K = len(topic_vocab)
    n_x = 5
    n_y = int(np.ceil(K / n_x))
    fig, axes = plt.subplots(n_y, n_x, figsize=(2.5 * n_x, 4 * n_y), sharex=True)
    axes = axes.flatten()
    for topic_idx, topic in enumerate(topic_vocab):
        top_features_ind = topic.argsort()[:-n_top_words - 1:-1]
        top_features = [feature_names[i] for i in top_features_ind]
        weights = topic[top_features_ind]

        ax = axes[topic_idx]
        ax.barh(top_features, weights, height=0.7)
        ax.set_title(f'Topic {topic_idx +1}',
                     fontdict={'fontsize': 14})
        ax.invert_yaxis()
        ax.tick_params(axis='both', which='major', labelsize=12)
        for i in 'top right left'.split():
            ax.spines[i].set_visible(False)
        fig.suptitle(title, fontsize=20)

    plt.subplots_adjust(top=0.90, bottom=0.05, wspace=0.90, hspace=0.3)
    plt.show()

In [ ]:
feature_names = [vocab.itos[i] for i in range(0, len(vocab))]

In [ ]:
k = 10
this_model = models_k[k]
decoder = this_model.decoder[0]
weights = decoder.weight.data.detach().clone().numpy()
topic_vocab_mat = weights.T

plot_top_words(topic_vocab_mat, feature_names, 10, f"NVDM K={k} Topics")

In [ ]:
def perplexity(model, dataset, n_sample):
    """ Compute perplexity of PyTorch model on the PyTorch Dataset. """

    # Iterate over the whole dataset once (1 big batch).
    data_loader = DataLoader(dataset,
                             batch_size=len(dataset),
                             shuffle=False,
                             collate_fn=collate_batch)
    assert len(data_loader) == 1

    for idx, (text, offsets) in enumerate(data_loader):
        text = text.to("cpu")
        offsets = offsets.to("cpu")

        with torch.no_grad():
            

            # Estimate the loss for each document by sampling
            loss_dict = this_model(text, offsets, kl_weight=1.0, n_sample=n_sample)
            loss = loss_dict["total"]

            # According to the NVDM paper, we estimate log probabilities using
            # the variational lower bound (negative of loss).
            log_probs = -loss

            # Perplexity below...
            
            # Hack to get the number of words per document
            # using the offsets (starting index of each document).
            doc_lengths = torch.zeros(offsets.size(0))
            doc_lengths[:-1] = offsets[1:] - offsets[:-1]
            doc_lengths[-1] = text.size(0) - offsets[-1]
            # # Sanity check
            # assert torch.all(torch.tensor([len(bd.split(" ")) for bd in BDs]) == doc_lengths)

            # Average over the words for each document
            ppx_doc = log_probs / doc_lengths

            # Perplexity: argument averages over documents
            ppx = torch.exp(-ppx_doc.mean()).item()

            return ppx

In [ ]:
# Compute perplexity example
k = 10
this_model = models_k[k]
this_model = this_model.to("cpu")
this_model.eval()

perplexity(this_model, dataset, 20)

# NVDM Stock Returns

In [ ]:
import pandas as pd
import os
import numpy as np
from matplotlib import pyplot as pp
from pathlib import Path

# from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
# from sklearn.decomposition import LatentDirichletAllocation
from sklearn.preprocessing import StandardScaler
import statsmodels.api as sm

In [ ]:
import pickle
from collections import Counter

from nltk.stem import PorterStemmer
from models import nvdm
import torch

## Business Description Data
### SP500 Data

In [ ]:
sp500_dir = Path('../SP500')
filenames = os.listdir(sp500_dir)
filenames[:5]

In [ ]:
# Load data
ticker_sp50 = []
sector_sp50 = []
bds_sp50 = []
for fn in filenames:
    fn_first = fn.split('.txt')[0]
    ticker, sector = fn_first.split('_')

    path = os.path.join(sp500_dir, fn)
    with open(path, 'r', encoding="utf8") as f:
        f_text = f.read()
    
    ticker_sp50.append(ticker)
    sector_sp50.append(sector)
    bds_sp50.append(f_text)

In [ ]:
# Basic sanity checks

# Just one file is named wrong
if ticker_sp50[19] == "VF Corporation":
    ticker_sp50[19] = "VFC"
assert all(s.isupper() for s in ticker_sp50)
assert all(s.title() == s for s in sector_sp50)
assert all (len(bd) > 0 for bd in bds_sp50)

### Larger Dataset - excluding SP500

In [ ]:
# NOTE: we use the cleaned version of bds_1.txt.
# Format is the same except the business descriptions are preprocessed.

f = open("./data/bds_1_clean.txt", "r", encoding="utf8")
f_lines = f.readlines()
f.close()

company_ids_all = f_lines[0::2]
company_descriptions_all = f_lines[1::2]
company_tickers = [x.split(':')[0] for x in company_ids_all]
removeSP = np.in1d(np.array(company_tickers), list(ticker_sp50))

bds_all = []
ticker_all = []
for i, d in enumerate(company_descriptions_all):
    if (len(d) > 3000) and not removeSP[i]:
        bds_all.append(d)
        ticker_all.append(company_ids_all[i])

## Get Returns Data

In [ ]:
start_d = np.datetime64('2018-01-01')
end_d = np.datetime64('2020-01-01')
business_ds = pd.date_range(start_d, end_d, freq='B')

price_data = pd.read_csv('./data/Price.csv')

select_these = np.in1d(price_data.tic.values, list(ticker_sp50))
price_sp50 = price_data.loc[select_these, ['tic', 'datadate', 'prccd']]
price_sp50['datadate'] = pd.to_datetime(price_sp50['datadate'], format='%Y%m%d')
price_sp50 = pd.pivot_table(price_sp50,index='datadate',columns='tic',values='prccd')
price_sp50 = price_sp50.ffill(limit=5)
price_sp50 = price_sp50.reindex(business_ds)
price_sp50 = price_sp50.dropna(axis=0)

returns_sp50 = np.log(price_sp50) - np.log(price_sp50.shift(1))
returns_sp50 = returns_sp50.dropna(axis=0)

## Explaining Returns with NVDM
### Get the pretrained NVDM torch models

In [ ]:
# Load vocabulary object.
with open("./vocabs/vocab_bds_1_clean_10000.pickle", "rb") as f:
    vocab = pickle.load(f)

In [ ]:
# Load pretrained NVDM models for different k.

MODELSAVE_PATH = "./modelsaves"

models_k = dict()
k_values = []

hidden_size = 500

for filename in os.listdir(MODELSAVE_PATH):
    
    num_topics = filename.split("_")[1][1:]
    num_topics = int(num_topics)
    k_values.append(num_topics)
    
    model = nvdm.NVDM(len(vocab), hidden_size, num_topics, 1, "cpu")
    model.load_state_dict(torch.load(os.path.join(MODELSAVE_PATH, filename), map_location="cpu"))
    model.eval()
    models_k[num_topics] = model
    
    # Hidden size was hard-coded in NVDM_Experiments, sanity check
    assert model.n_hidden == hidden_size

In [ ]:
models_k[10]

## Get Features for S&P 500
More manual work needed for NVDM (no fit/transform() functions)

In [ ]:
def nvdm_transform(model, bow):
    # replacing this_lda.transform(tf_sp50)
    vocab_topic_mat = model.decoder[0].weight.data.detach().clone().numpy()
    doc_topic_mat = bow @ vocab_topic_mat
    return doc_topic_mat

In [ ]:
# Make the BOW matrix manually, using the existing Vocab's token-to-index mapping.
bow_sp50 = np.zeros((len(bds_sp50), len(vocab)))

for d, bd in enumerate(bds_sp50):
    token_idxs = vocab.lookup_indices(bd.split(" "))
    word_counts = Counter(token_idxs)
    for w, count in word_counts.items():
        bow_sp50[d, w] = count   
bow_sp50.shape

In [ ]:
# Get the learned semantic embeddings from a model (e.g. K=10)
vocab_topic_mat = models_k[10].decoder[0].weight.data.detach().clone().numpy()
vocab_topic_mat.shape

In [ ]:
# This should represent each document as a linear combination of its word embeddings.
# i.e. for each document, sum its word embeddings.
doc_topic_mat = bow_sp50 @ vocab_topic_mat
doc_topic_mat.shape

In [ ]:
# Continue the same way as LDA...
features_sp50 = doc_topic_mat
features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)

### Loop Over Dates and Perform OLS Regression

In [ ]:
all_dates = returns_sp50.index
max_f = 19
adj_r2_features = []

for dd in all_dates:
    reg_data = returns_sp50.loc[[dd]].transpose().join(features_sp50_df.loc[:, 0:max_f]).dropna(axis=0).values
    y = reg_data[:, 0]
    X = reg_data[:, 1:]

    std_scaler = StandardScaler()
    X = std_scaler.fit_transform(X)

    X = sm.add_constant(X, prepend=False)
    ols_model = sm.OLS(y, X)
    res = ols_model.fit()
    adj_r2_features.append(res.rsquared_adj)

In [ ]:
pp.plot(all_dates, adj_r2_features)

### Comparing Different K

Next, calculate average regression adjusted $R^2$ for different K

In [ ]:
tf_sp50 = bow_sp50
all_ks = list(sorted(models_k.keys()))

all_adj_r2 = []
for k in all_ks:
    print(f'Running for k = {k}')
    
    features_sp50 = nvdm_transform(models_k[k], bow_sp50)
    features_sp50_df = pd.DataFrame(index=ticker_sp50, data=features_sp50)
    adj_r2_dates = []
    for dd in all_dates:
        reg_data = returns_sp50.loc[[dd]].transpose().join(features_sp50_df.loc[:, 0:max_f]).dropna(axis=0).values
        y = reg_data[:, 0]
        X = reg_data[:, 1:]

        std_scaler = StandardScaler()
        X = std_scaler.fit_transform(X)

        X = sm.add_constant(X, prepend=False)
        ols_model = sm.OLS(y, X)
        res = ols_model.fit()
        adj_r2_dates.append(res.rsquared_adj)
    
    all_adj_r2.append(np.mean(adj_r2_dates))

In [ ]:
all_adj_r2